# Hierarchically Deep Convolutional Neural Network For Image Recognition

## Setup and Imports

**Import Packages**

In [1]:
import keras as kr
import numpy as np
import tensorflow as tf

from keras.datasets import cifar100

from sklearn.model_selection import train_test_split

from random import randint
import time

Using TensorFlow backend.


**Define Global Variables**

In [2]:
# The number of coarse categories
coarse_categories = 20

# The number of fine categories
fine_categories = 100

## Import and Preprocess Dataset

**Import Cifar100 Data Set**

In [3]:
(X, y_c), (x_test, y_c_test) = cifar100.load_data(label_mode='coarse')
(X, y), (x_test, y_test) = cifar100.load_data(label_mode='fine')

**Fine-To-Coarse Mapping**

(Ideally, this would be done through spectral clustering as opposed to hard-coding)

In [4]:
fine2coarse = np.zeros((fine_categories,coarse_categories))
for i in range(coarse_categories):
    index = np.where(y_c_test[:,0] == i)[0]
    fine_cat = np.unique([y_test[j,0] for j in index])
    for j in fine_cat:
        fine2coarse[j,i] = 1


In [5]:
y_c = 0; # Clear y_c in interest of saving mem
y_c_test=0;

In [6]:
################################################################################
#    Title: One Hot Encoding
################################################################################
#    Description: 
#        This function extends a matrix to one-hot encoding
#    
#    Parameters:
#        y    Array of label values
# 
#    Returns:
#        y_new    One hot encoded array of labels
################################################################################
def one_hot(y):
    n_values = np.max(y) + 1
    y_new = np.eye(n_values)[y[:,0]]
    return y_new

In [7]:
y=one_hot(y)
y_test=one_hot(y_test)
print(np.shape(y))

(50000, 100)


**Apply ZCA Whitening**

In [8]:
################################################################################
#    Title: ZCA
################################################################################
#    Description: 
#        This function applies ZCA Whitening to the image set
#    
#    Parameters:
#        x_1           Array of MxNxC images to compute the ZCA Whitening
#        x_2           Array of MxNxC images to apply the ZCA transform
#        num_batch    Number of batches to do the computation
# 
#    Returns:
#        An array of MxNxC zca whitened images
################################################################################
def zca(x_1, x_2, epsilon=1e-5):
        
    with tf.name_scope('ZCA'):
        
        x1 = tf.placeholder(tf.float64, shape=np.shape(x_1), name='placeholder_x1')
        x2 = tf.placeholder(tf.float64, shape=np.shape(x_2), name='placeholder_x2')
        
        flatx = tf.cast(tf.reshape(x1, (-1, np.prod(x_1.shape[-3:])),name="reshape_flat"),tf.float64,name="flatx")
        sigma = tf.tensordot(tf.transpose(flatx),flatx, 1,name="sigma") / tf.cast(tf.shape(flatx)[0],tf.float64) ### N-1 or N?
        s, u, v = tf.svd(sigma,name="svd")
        pc = tf.tensordot(tf.tensordot(u,tf.diag(1. / tf.sqrt(s+epsilon)),1,name="inner_dot"),tf.transpose(u),1, name="pc")
        
        net1 = tf.tensordot(flatx, pc,1,name="whiten1")
        net1 = tf.reshape(net1,np.shape(x_1), name="output1")
        
        flatx2 = tf.cast(tf.reshape(x2, (-1, np.prod(x_2.shape[-3:])),name="reshape_flat2"),tf.float64,name="flatx2")
        net2 = tf.tensordot(flatx2, pc,1,name="whiten2")
        net2 = tf.reshape(net2,np.shape(x_2), name="output2")
        
    with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            x_1,x_2 = sess.run([net1,net2], feed_dict={x1: x_1, x2: x_2})    
    return x_1,x_2

In [9]:
time1 = time.time()
X,x_test = zca(X,x_test)
time2 = time.time()
print('Time Elapsed - ZCA Whitening: '+str(time2-time1));

Time Elapsed - ZCA Whitening: 47.44490122795105


**Split Training set into Training and Validation sets**

In [10]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=.1, random_state=0)
X = 0
y = 0

**Flip, pad and randomly crop each photo**

In [11]:
################################################################################
#    Title: Preprocess Img
################################################################################
#    Description: 
#        This function pads images by 4 pixels, randomly crops them, then
#        randomly flips them
#    
#    Parameters:
#        x_1           Array of MxNxC images to compute the ZCA Whitening
#        x_2           Array of MxNxC images to apply the ZCA transform
#        num_batch    Number of batches to do the computation
# 
#    Returns:
#        An array of MxNxC zca whitened images
################################################################################
def preprocess_img(X,y):
        
    with tf.name_scope('Preproc'):
        
        images = tf.placeholder(tf.float64, shape=np.shape(X))
        labels = tf.placeholder(tf.float64, shape=np.shape(y))
        
        net = tf.map_fn(lambda img: tf.image.flip_left_right(img), images)
        net = tf.map_fn(lambda img: tf.image.rot90(img), net)
        net = tf.image.resize_image_with_crop_or_pad(net,40,40)
        net = tf.map_fn(lambda img: tf.random_crop(img, [32,32,3]), net)

        net1 = tf.image.resize_image_with_crop_or_pad(images,40,40)
        net1 = tf.map_fn(lambda img: tf.random_crop(img, [32,32,3]), net1)
        
        net = tf.concat([net, net1],0)
        net = tf.random_shuffle(net, seed=0)
        net_labels = tf.concat([labels, labels],0)
        net_labels = tf.random_shuffle(net_labels,seed=0)
        
        net = tf.map_fn(lambda img: tf.image.random_flip_up_down(img), net)
        
    with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            x_t,y_t = sess.run([net,net_labels], feed_dict={images: X, labels: y})    
    return x_t,y_t


In [12]:
time1 = time.time()
x_train,y_train = preprocess_img(x_train,y_train)
time2 = time.time()
print('Time Elapsed - Img Preprocessing: '+str(time2-time1));

Time Elapsed - Img Preprocessing: 66.82172513008118


## Single Classifier Training

**Constructing CNN**

In [19]:
from keras import optimizers
from keras.layers import Input, Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from keras.models import Model

in_layer = Input(shape=(32, 32, 3), dtype='float32', name='main_input')

net = Conv2D(384, 3, strides=1, padding='same', activation='elu')(in_layer)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(384, 1, strides=1, padding='same', activation='elu')(net)
net = Conv2D(384, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(640, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(640, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.2)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(640, 1, strides=1, padding='same', activation='elu')(net)
net = Conv2D(768, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(768, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(768, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.3)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(768, 1, strides=1, padding='same', activation='elu')(net)
net = Conv2D(896, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(896, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.4)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(896, 3, strides=1, padding='same', activation='elu')(net)
net = Conv2D(1024, 2, strides=1, padding='same', activation='elu')(net)
net = Conv2D(1024, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.5)(net)
net = MaxPooling2D((2, 2), padding='valid')(net)

net = Conv2D(1024, 1, strides=1, padding='same', activation='elu')(net)
net = Conv2D(1152, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.6)(net)
net = MaxPooling2D((2, 2), padding='same')(net)

net = Flatten()(net)
net = Dense(1152, activation='elu')(net)
net = Dense(100, activation='softmax')(net)

**Compile Model**

In [20]:
model = Model(inputs=in_layer,outputs=net)
sgd_coarse = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer= sgd_coarse, loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
#model.load_weights('data/models/model_coarse'+str(30))

**Train Model**

In [22]:
tbCallBack = kr.callbacks.TensorBoard(log_dir='./data/graph/elu_drop/', histogram_freq=0, write_graph=True, write_images=True)

In [23]:
batch = 64

In [24]:
index= 0
step = 5
stop = 30

while index < stop:
    model.fit(x_train, y_train, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val), callbacks=[tbCallBack])
    index += step
    model.save_weights('data/models/model_coarse'+str(index))
save_index = index

Train on 90000 samples, validate on 5000 samples
Epoch 1/5
90000/90000 [==============================] - 702s - loss: 4.1678 - acc: 0.0558 - val_loss: 3.9135 - val_acc: 0.1132
Epoch 2/5
90000/90000 [==============================] - 699s - loss: 3.3947 - acc: 0.1657 - val_loss: 3.8714 - val_acc: 0.2070
Epoch 3/5
90000/90000 [==============================] - 699s - loss: 3.0618 - acc: 0.2287 - val_loss: 4.5739 - val_acc: 0.2350
Epoch 4/5
90000/90000 [==============================] - 699s - loss: 2.8058 - acc: 0.2842 - val_loss: 5.4959 - val_acc: 0.2396
Epoch 5/5
90000/90000 [==============================] - 699s - loss: 2.6057 - acc: 0.3282 - val_loss: 5.3397 - val_acc: 0.2920
Train on 90000 samples, validate on 5000 samples
Epoch 6/10
90000/90000 [==============================] - 700s - loss: 2.4349 - acc: 0.3679 - val_loss: 5.5889 - val_acc: 0.3210
Epoch 7/10
90000/90000 [==============================] - 700s - loss: 2.2847 - acc: 0.4041 - val_loss: 4.1056 - val_acc: 0.3252
Epoc

### Load Most Recent Model

In [25]:
sgd_fine = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

In [26]:
for i in range(len(model.layers)):
    model.layers[i].trainable=False

## Fine-Tuning for Coarse Classifier

In [27]:
y_train_c = np.dot(y_train,fine2coarse)
y_val_c = np.dot(y_val,fine2coarse)

In [28]:
net = Conv2D(1024, 1, strides=1, padding='same', activation='elu')(model.layers[-8].output)
net = Conv2D(1152, 2, strides=1, padding='same', activation='elu')(net)
net = Dropout(.6)(net)
net = MaxPooling2D((2, 2), padding='same')(net)

net = Flatten()(net)
net = Dense(1152, activation='elu')(net)
out_coarse = Dense(20, activation='softmax')(net)

model_c = Model(inputs=in_layer,outputs=out_coarse)
model_c.compile(optimizer= sgd_coarse, loss='categorical_crossentropy', metrics=['accuracy'])

for i in range(len(model_c.layers)-1):
    model_c.layers[i].set_weights(model.layers[i].get_weights())

In [29]:
index = 30
step = 10
stop = 40

while index < stop:
    model_c.fit(x_train, y_train_c, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val_c), callbacks=[tbCallBack])
    index += step

Train on 90000 samples, validate on 5000 samples
Epoch 31/40
90000/90000 [==============================] - 230s - loss: 1.2687 - acc: 0.6486 - val_loss: 3.8728 - val_acc: 0.5522
Epoch 32/40
90000/90000 [==============================] - 232s - loss: 1.1943 - acc: 0.6648 - val_loss: 3.8772 - val_acc: 0.5520
Epoch 33/40
90000/90000 [==============================] - 231s - loss: 1.1875 - acc: 0.6667 - val_loss: 3.7064 - val_acc: 0.5654
Epoch 34/40
90000/90000 [==============================] - 231s - loss: 1.1720 - acc: 0.6689 - val_loss: 3.7129 - val_acc: 0.5610
Epoch 35/40
90000/90000 [==============================] - 232s - loss: 1.1669 - acc: 0.6715 - val_loss: 3.6124 - val_acc: 0.5646
Epoch 36/40
90000/90000 [==============================] - 231s - loss: 1.1672 - acc: 0.6713 - val_loss: 3.7043 - val_acc: 0.5558
Epoch 37/40
90000/90000 [==============================] - 231s - loss: 1.1556 - acc: 0.6748 - val_loss: 3.7125 - val_acc: 0.5440
Epoch 38/40
90000/90000 [================

In [30]:
model_c.compile(optimizer=sgd_fine, loss='categorical_crossentropy', metrics=['accuracy'])
stop = 50

while index < stop:
    model_c.fit(x_train, y_train_c, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_val, y_val_c), callbacks=[tbCallBack])
    index += step

Train on 90000 samples, validate on 5000 samples
Epoch 41/50
90000/90000 [==============================] - 231s - loss: 1.0175 - acc: 0.7034 - val_loss: 3.3435 - val_acc: 0.5900
Epoch 42/50
90000/90000 [==============================] - 231s - loss: 0.9962 - acc: 0.7072 - val_loss: 3.3375 - val_acc: 0.5892
Epoch 43/50
90000/90000 [==============================] - 231s - loss: 0.9812 - acc: 0.7106 - val_loss: 3.3343 - val_acc: 0.5922
Epoch 44/50
90000/90000 [==============================] - 232s - loss: 0.9842 - acc: 0.7113 - val_loss: 3.3335 - val_acc: 0.5882
Epoch 45/50
90000/90000 [==============================] - 231s - loss: 0.9725 - acc: 0.7118 - val_loss: 3.3180 - val_acc: 0.5854
Epoch 46/50
90000/90000 [==============================] - 231s - loss: 0.9668 - acc: 0.7142 - val_loss: 3.2924 - val_acc: 0.5854
Epoch 47/50
90000/90000 [==============================] - 231s - loss: 0.9676 - acc: 0.7132 - val_loss: 3.2755 - val_acc: 0.5900
Epoch 48/50
90000/90000 [================

## Fine-Tuning for Fine Classifiers

### Construct Fine Classifiers

In [35]:
def fine_model():
    net = Conv2D(1024, 1, strides=1, padding='same', activation='elu')(model.layers[-8].output)
    net = Conv2D(1152, 2, strides=1, padding='same', activation='elu')(net)
    net = Dropout(.6)(net)
    net = MaxPooling2D((2, 2), padding='same')(net)

    net = Flatten()(net)
    net = Dense(1152, activation='elu')(net)
    out_fine = Dense(100, activation='softmax')(net)
    model_fine = Model(inputs=in_layer,outputs=out_fine)
    model_fine.compile(optimizer= sgd_coarse,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    for i in range(len(model_fine.layers)-1):
        model_fine.layers[i].set_weights(model.layers[i].get_weights())
    return model_fine

In [36]:
fine_models = {'models' : [{} for i in range(coarse_categories)], 'yhf' : [{} for i in range(coarse_categories)]}
for i in range(coarse_categories):
    model_i = fine_model()
    fine_models['models'][i] = model_i

### Train Fine Classifiers on Respective Data

In [37]:
def get_error(y,yh):
    # Threshold 
    yht = np.zeros(np.shape(yh))
    yht[np.arange(len(yh)), yh.argmax(1)] = 1
    # Evaluate Error
    error = np.count_nonzero(np.count_nonzero(y-yht,1))/len(y)
    return error

In [38]:
for i in range(coarse_categories):
    index= 0
    step = 5
    stop = 5
    
    # Get all training data for the coarse category
    ix = np.where([(y_train[:,j]==1) for j in [k for k, e in enumerate(fine2coarse[:,i]) if e != 0]])[1]
    x_tix = x_train[ix]
    y_tix = y_train[ix]
    
    # Get all validation data for the coarse category
    ix_v = np.where([(y_val[:,j]==1) for j in [k for k, e in enumerate(fine2coarse[:,i]) if e != 0]])[1]
    x_vix = x_val[ix_v]
    y_vix = y_val[ix_v]
    
    while index < stop:
        fine_models['models'][i].fit(x_tix, y_tix, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_vix, y_vix))
        index += step
    
    fine_models['models'][i].compile(optimizer=sgd_fine, loss='categorical_crossentropy', metrics=['accuracy'])
    stop = 10

    while index < stop:
        fine_models['models'][i].fit(x_tix, y_tix, batch_size=batch, initial_epoch=index, epochs=index+step, validation_data=(x_vix, y_vix))
        index += step
        
    yh_f = fine_models['models'][i].predict(x_val[ix_v], batch_size=batch)
    print('Fine Classifier '+str(i)+' Error: '+str(get_error(y_val[ix_v],yh_f))) 

Train on 4524 samples, validate on 238 samples
Epoch 1/5
4524/4524 [==============================] - 13s - loss: 1.3681 - acc: 0.5115 - val_loss: 5.4511 - val_acc: 0.3445
Epoch 2/5
4524/4524 [==============================] - 11s - loss: 1.2362 - acc: 0.5511 - val_loss: 5.4643 - val_acc: 0.3235
Epoch 3/5
4524/4524 [==============================] - 11s - loss: 1.1453 - acc: 0.5681 - val_loss: 5.6825 - val_acc: 0.3655
Epoch 4/5
4524/4524 [==============================] - 11s - loss: 1.1168 - acc: 0.5829 - val_loss: 5.4719 - val_acc: 0.3613
Epoch 5/5
4524/4524 [==============================] - 11s - loss: 1.1665 - acc: 0.5584 - val_loss: 5.5140 - val_acc: 0.3571
Train on 4524 samples, validate on 238 samples
Epoch 6/10
4524/4524 [==============================] - 12s - loss: 0.9453 - acc: 0.6143 - val_loss: 5.3983 - val_acc: 0.3655
Epoch 7/10
4524/4524 [==============================] - 11s - loss: 0.9243 - acc: 0.6165 - val_loss: 5.3284 - val_acc: 0.3655
Epoch 8/10
4524/4524 [=======

4478/4478 [==============================] - 11s - loss: 0.4718 - acc: 0.8403 - val_loss: 3.1810 - val_acc: 0.6935
Epoch 10/10
4478/4478 [==============================] - 11s - loss: 0.4544 - acc: 0.8399 - val_loss: 3.1589 - val_acc: 0.6935
Fine Classifier 5 Error: 0.3065134099616858
Train on 4450 samples, validate on 275 samples
Epoch 1/5
4450/4450 [==============================] - 13s - loss: 1.0950 - acc: 0.6564 - val_loss: 6.0110 - val_acc: 0.4618
Epoch 2/5
4450/4450 [==============================] - 11s - loss: 0.9277 - acc: 0.6865 - val_loss: 5.9392 - val_acc: 0.4655
Epoch 3/5
4450/4450 [==============================] - 11s - loss: 0.8500 - acc: 0.7007 - val_loss: 5.9275 - val_acc: 0.4764
Epoch 4/5
4450/4450 [==============================] - 11s - loss: 0.8522 - acc: 0.7034 - val_loss: 5.8887 - val_acc: 0.4618
Epoch 5/5
4450/4450 [==============================] - 11s - loss: 0.8672 - acc: 0.6989 - val_loss: 6.0145 - val_acc: 0.4764
Train on 4450 samples, validate on 275 sam

4540/4540 [==============================] - 11s - loss: 0.7471 - acc: 0.7238 - val_loss: 6.1818 - val_acc: 0.4043
Epoch 8/10
4540/4540 [==============================] - 11s - loss: 0.7719 - acc: 0.7240 - val_loss: 6.1484 - val_acc: 0.4261
Epoch 9/10
4540/4540 [==============================] - 11s - loss: 0.7305 - acc: 0.7311 - val_loss: 6.1477 - val_acc: 0.4174
Epoch 10/10
4540/4540 [==============================] - 11s - loss: 0.7423 - acc: 0.7300 - val_loss: 6.1217 - val_acc: 0.4261
Fine Classifier 11 Error: 0.5739130434782609
Train on 4502 samples, validate on 249 samples
Epoch 1/5
4502/4502 [==============================] - 15s - loss: 0.7717 - acc: 0.7654 - val_loss: 4.9607 - val_acc: 0.5141
Epoch 2/5
4502/4502 [==============================] - 11s - loss: 0.5712 - acc: 0.8127 - val_loss: 5.1192 - val_acc: 0.4980
Epoch 3/5
4502/4502 [==============================] - 11s - loss: 0.5284 - acc: 0.8152 - val_loss: 4.8457 - val_acc: 0.5181
Epoch 4/5
4502/4502 [==================

4462/4462 [==============================] - 12s - loss: 0.8677 - acc: 0.6797 - val_loss: 3.6947 - val_acc: 0.5465
Epoch 7/10
4462/4462 [==============================] - 11s - loss: 0.8332 - acc: 0.6842 - val_loss: 3.7074 - val_acc: 0.5502
Epoch 8/10
4462/4462 [==============================] - 11s - loss: 0.8291 - acc: 0.6934 - val_loss: 3.6796 - val_acc: 0.5204
Epoch 9/10
4462/4462 [==============================] - 11s - loss: 0.8464 - acc: 0.6806 - val_loss: 3.7036 - val_acc: 0.5167
Epoch 10/10
4462/4462 [==============================] - 11s - loss: 0.8339 - acc: 0.6851 - val_loss: 3.7270 - val_acc: 0.5130
Fine Classifier 17 Error: 0.48698884758364314
Train on 4492 samples, validate on 254 samples
Epoch 1/5
4492/4492 [==============================] - 13s - loss: 0.7663 - acc: 0.7567 - val_loss: 3.7631 - val_acc: 0.5866
Epoch 2/5
4492/4492 [==============================] - 11s - loss: 0.5795 - acc: 0.7974 - val_loss: 3.8636 - val_acc: 0.5866
Epoch 3/5
4492/4492 [================

## Probabilistic Averaging

In [45]:
def eval_hdcnn(X, y):
    yh = np.zeros(np.shape(y))
    
    yh_s = model.predict(X, batch_size=batch)
    
    print('Single Classifier Error: '+str(get_error(y,yh_s)))
    
    yh_c = model_c.predict(X, batch_size=batch)
    y_c = np.dot(y,fine2coarse)
    
    print('Coarse Classifier Error: '+str(get_error(y_c,yh_c)))

    for i in range(coarse_categories):
        if i%5 == 0:
            print("Evaluating Fine Classifier: ", str(i))
        #fine_models['yhf'][i] = fine_models['models'][i].predict(X, batch_size=batch)
        yh += np.multiply(yh_c[:,i].reshape((len(y)),1), fine_models['yhf'][i])
    
    print('Overall Error: '+str(get_error(y,yh)))
    return yh

In [46]:
yh = eval_hdcnn(x_val,y_val)

Single Classifier Error: 0.6574
Coarse Classifier Error: 0.4126
Evaluating Fine Classifier:  0
Evaluating Fine Classifier:  5
Evaluating Fine Classifier:  10
Evaluating Fine Classifier:  15
Overall Error: 0.604
